In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## ধাপ ১: কাঠামোবদ্ধ আউটপুটের জন্য Pydantic মডেল সংজ্ঞায়িত করুন

এই মডেলগুলো এজেন্টদের ফেরত দেওয়া **স্কিমা** নির্ধারণ করে। Pydantic-এর সাথে `response_format` ব্যবহার করলে নিশ্চিত হয়:
- ✅ টাইপ-নিরাপদ ডেটা নিষ্কাশন
- ✅ স্বয়ংক্রিয় যাচাইকরণ
- ✅ ফ্রি-টেক্সট প্রতিক্রিয়া থেকে কোনো পার্সিং ত্রুটি নেই
- ✅ ক্ষেত্রগুলোর উপর ভিত্তি করে সহজ শর্তাধীন রাউটিং


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## ধাপ ২: হোটেল বুকিং টুল তৈরি করুন

এই টুলটি **availability_agent** ব্যবহার করবে রুম উপলব্ধ কিনা তা পরীক্ষা করার জন্য। আমরা `@ai_function` ডেকোরেটর ব্যবহার করি:
- একটি পাইথন ফাংশনকে AI-কলযোগ্য টুলে রূপান্তর করতে
- LLM-এর জন্য স্বয়ংক্রিয়ভাবে JSON স্কিমা তৈরি করতে
- প্যারামিটার যাচাই পরিচালনা করতে
- এজেন্টদের দ্বারা স্বয়ংক্রিয় আহ্বান সক্ষম করতে

এই ডেমোর জন্য:
- **স্টকহোম, সিয়াটল, টোকিও, লন্ডন, আমস্টারডাম** → রুম আছে ✅
- **অন্যান্য সব শহর** → রুম নেই ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## ধাপ ৩: রাউটিংয়ের জন্য শর্ত ফাংশন সংজ্ঞায়িত করুন

এই ফাংশনগুলো এজেন্টের প্রতিক্রিয়া পরীক্ষা করে এবং কর্মপ্রবাহে কোন পথ নেওয়া হবে তা নির্ধারণ করে।

**মূল প্যাটার্ন:**
1. বার্তাটি `AgentExecutorResponse` কিনা তা পরীক্ষা করুন
2. গঠিত আউটপুট (Pydantic মডেল) বিশ্লেষণ করুন
3. রাউটিং নিয়ন্ত্রণ করতে `True` বা `False` ফেরত দিন

কর্মপ্রবাহ এই শর্তগুলোকে **এজগুলোর** উপর মূল্যায়ন করবে, পরবর্তী কোন এক্সিকিউটরকে আহ্বান করা হবে তা সিদ্ধান্ত নিতে।


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## ধাপ ৪: কাস্টম ডিসপ্লে এক্সিকিউটর তৈরি করুন

এক্সিকিউটর হল ওয়ার্কফ্লো উপাদান যা রূপান্তর বা পার্শ্বপ্রভাব সম্পাদন করে। আমরা `@executor` ডেকোরেটর ব্যবহার করে একটি কাস্টম এক্সিকিউটর তৈরি করি যা চূড়ান্ত ফলাফল প্রদর্শন করে।

**মূল ধারণা:**
- `@executor(id="...")` - একটি ফাংশনকে ওয়ার্কফ্লো এক্সিকিউটর হিসেবে নিবন্ধন করে
- `WorkflowContext[Never, str]` - ইনপুট/আউটপুটের জন্য টাইপ হিন্ট
- `ctx.yield_output(...)` - চূড়ান্ত ওয়ার্কফ্লো ফলাফল প্রদান করে


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## ধাপ ৫: পরিবেশ ভেরিয়েবল লোড করুন

LLM ক্লায়েন্ট কনফিগার করুন। এই উদাহরণটি কাজ করে:
- **GitHub মডেলসমূহ** (GitHub টোকেন সহ ফ্রি টিয়ার)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## ধাপ ৬: কাঠামোবদ্ধ আউটপুট সহ AI এজেন্ট তৈরি করুন

আমরা **তিনটি বিশেষায়িত এজেন্ট** তৈরি করি, প্রতিটি `AgentExecutor` দিয়ে মোড়ানো:

1. **availability_agent** - টুল ব্যবহার করে হোটেলের প্রাপ্যতা পরীক্ষা করে
2. **alternative_agent** - বিকল্প শহর প্রস্তাব করে (যখন কোনো রুম নেই)
3. **booking_agent** - বুকিং উৎসাহিত করে (যখন রুম পাওয়া যায়)

**মূল বৈশিষ্ট্যসমূহ:**
- `tools=[hotel_booking]` - এজেন্টকে টুল সরবরাহ করে
- `response_format=PydanticModel` - কাঠামোবদ্ধ JSON আউটপুট নিশ্চিত করে
- `AgentExecutor(..., id="...")` - এজেন্টকে ওয়ার্কফ্লো ব্যবহারের জন্য মোড়ায়


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## ধাপ ৭: শর্তযুক্ত প্রান্ত দিয়ে ওয়ার্কফ্লো তৈরি করুন

এখন আমরা `WorkflowBuilder` ব্যবহার করে শর্তযুক্ত রাউটিং সহ গ্রাফ তৈরি করব:

**ওয়ার্কফ্লো কাঠামো:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**মূল পদ্ধতিগুলো:**
- `.set_start_executor(...)` - প্রবেশ বিন্দু নির্ধারণ করে
- `.add_edge(from, to, condition=...)` - শর্তযুক্ত প্রান্ত যোগ করে
- `.build()` - ওয়ার্কফ্লো চূড়ান্ত করে


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## ধাপ ৮: টেস্ট কেস ১ চালান - শহর যেখানে রুম নেই (প্যারিস)

চলুন **কোনো রুম নেই** এই পরিস্থিতি পরীক্ষা করি, প্যারিসে হোটেল অনুরোধ করে (যেখানে আমাদের সিমুলেশনে কোনো রুম নেই)।


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## ধাপ ৯: টেস্ট কেস ২ চালান - উপলব্ধতা সহ শহর (স্টকহোম)

এখন আসুন **উপলব্ধতা** পথটি পরীক্ষা করি, স্টকহোমে হোটেল অনুরোধ করে (যেখানে আমাদের সিমুলেশনে রুম রয়েছে)।


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## মূল বিষয় এবং পরবর্তী পদক্ষেপ

### ✅ আপনি যা শিখেছেন:

1. **WorkflowBuilder প্যাটার্ন**
   - `.set_start_executor()` ব্যবহার করে এন্ট্রি পয়েন্ট নির্ধারণ করুন
   - `.add_edge(from, to, condition=...)` ব্যবহার করে শর্তসাপেক্ষ রাউটিং করুন
   - `.build()` কল করে ওয়ার্কফ্লো চূড়ান্ত করুন

2. **শর্তসাপেক্ষ রাউটিং**
   - শর্ত ফাংশন `AgentExecutorResponse` পরিদর্শন করে
   - কাঠামোবদ্ধ আউটপুট বিশ্লেষণ করে রাউটিং সিদ্ধান্ত নিন
   - একটি এজ সক্রিয় করতে `True` ফেরত দিন, এড়িয়ে যেতে `False` ফেরত দিন

3. **টুল ইন্টিগ্রেশন**
   - Python ফাংশনকে AI টুলে রূপান্তর করতে `@ai_function` ব্যবহার করুন
   - এজেন্ট প্রয়োজন হলে স্বয়ংক্রিয়ভাবে টুল কল করে
   - টুল JSON ফেরত দেয় যা এজেন্ট বিশ্লেষণ করতে পারে

4. **কাঠামোবদ্ধ আউটপুট**
   - টাইপ-নিরাপদ ডেটা এক্সট্রাকশনের জন্য Pydantic মডেল ব্যবহার করুন
   - এজেন্ট তৈরি করার সময় `response_format=MyModel` সেট করুন
   - `Model.model_validate_json()` দিয়ে রেসপন্স বিশ্লেষণ করুন

5. **কাস্টম এক্সিকিউটর**
   - ওয়ার্কফ্লো কম্পোনেন্ট তৈরি করতে `@executor(id="...")` ব্যবহার করুন
   - এক্সিকিউটর ডেটা রূপান্তর করতে বা সাইড ইফেক্ট সম্পাদন করতে পারে
   - ওয়ার্কফ্লো ফলাফল উৎপাদন করতে `ctx.yield_output()` ব্যবহার করুন

### 🚀 বাস্তব জীবনের প্রয়োগ:

- **ট্রাভেল বুকিং**: উপলব্ধতা পরীক্ষা, বিকল্প প্রস্তাব, অপশন তুলনা
- **কাস্টমার সার্ভিস**: সমস্যা টাইপ, অনুভূতি, অগ্রাধিকার অনুযায়ী রাউটিং
- **ই-কমার্স**: ইনভেন্টরি পরীক্ষা, বিকল্প প্রস্তাব, অর্ডার প্রক্রিয়া
- **কন্টেন্ট মডারেশন**: টক্সিসিটি স্কোর, ইউজার ফ্ল্যাগ অনুযায়ী রাউটিং
- **অনুমোদন ওয়ার্কফ্লো**: পরিমাণ, ইউজার রোল, ঝুঁকির স্তর অনুযায়ী রাউটিং
- **মাল্টি-স্টেজ প্রসেসিং**: ডেটার গুণমান, সম্পূর্ণতার ভিত্তিতে রাউটিং

### 📚 পরবর্তী পদক্ষেপ:

- আরও জটিল শর্ত যোগ করুন (একাধিক মানদণ্ড)
- ওয়ার্কফ্লো স্টেট ম্যানেজমেন্ট দিয়ে লুপ বাস্তবায়ন করুন
- পুনর্ব্যবহারযোগ্য কম্পোনেন্টের জন্য সাব-ওয়ার্কফ্লো যোগ করুন
- বাস্তব API (হোটেল বুকিং, ইনভেন্টরি সিস্টেম) এর সাথে ইন্টিগ্রেট করুন
- ত্রুটি পরিচালনা এবং ফালব্যাক পাথ যোগ করুন
- বিল্ট-ইন ভিজ্যুয়ালাইজেশন টুল দিয়ে ওয়ার্কফ্লো ভিজ্যুয়ালাইজ করুন



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসাধ্য সঠিকতার জন্য চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়বদ্ধ থাকব না।
